# HR NET

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Bottleneck Block for the HRNet model
def bottleneck_block(x, filters, stride=1):
    shortcut = x
    # Main branch
    x = layers.Conv2D(filters // 4, 1, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters // 4, 3, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, 1, use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Shortcut branch
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride, use_bias=False)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

# High-Resolution Block
def high_resolution_block(x, filters, num_blocks):
    for _ in range(num_blocks):
        x = bottleneck_block(x, filters)
    return x

# Build the HRNet Model
def build_hrnet(input_shape=(180, 180, 3), num_classes=8):
    inputs = layers.Input(shape=input_shape)

    # Stem Stage
    x = layers.Conv2D(64, 3, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(64, 3, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Stage 1
    x = high_resolution_block(x, 256, num_blocks=4)

    # Classification Head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, x)
    return model

# Instantiate and compile the HRNet model
model = build_hrnet()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 180, 180, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 90, 90, 64)   1728        ['input_2[0][0]']                
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 90, 90, 64)  256         ['conv2d_4[0][0]']               
 rmalization)                                                                                     
                                                                                            

In [4]:
Batch_size = 32

# Data augmentation and preprocessing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load and prepare the data with square dimensions and data augmentation
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(180, 180),
    batch_size=Batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(180, 180),
    batch_size=Batch_size,
    class_mode='categorical'
)


Found 9376 images belonging to 8 classes.
Found 2344 images belonging to 8 classes.


In [5]:

# Train the HRNet model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // Batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // Batch_size
)

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Validation Accuracy:", accuracy)


Epoch 1/10
293/293 [==============================] - 345s 1s/step - loss: 0.9881 - accuracy: 0.6521 - val_loss: 1.9392 - val_accuracy: 0.0646
Epoch 2/10
293/293 [==============================] - 354s 1s/step - loss: 0.9102 - accuracy: 0.6701 - val_loss: 1.0636 - val_accuracy: 0.6618
Epoch 3/10
293/293 [==============================] - 544s 2s/step - loss: 0.8918 - accuracy: 0.6800 - val_loss: 0.9999 - val_accuracy: 0.6674
Epoch 4/10
293/293 [==============================] - 626s 2s/step - loss: 0.8619 - accuracy: 0.6860 - val_loss: 1.1951 - val_accuracy: 0.6520
Epoch 5/10
293/293 [==============================] - 611s 2s/step - loss: 0.8462 - accuracy: 0.6912 - val_loss: 1.7222 - val_accuracy: 0.3943
Epoch 6/10
293/293 [==============================] - 608s 2s/step - loss: 0.8088 - accuracy: 0.7079 - val_loss: 1.4397 - val_accuracy: 0.6421
Epoch 7/10
293/293 [==============================] - 605s 2s/step - loss: 0.7873 - accuracy: 0.7150 - val_loss: 1.0424 - val_accuracy: 0.6537

In [6]:
model.save("HRnet_custom.h5")

# Custom simple model

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

class MultiHeadAttentionLayer(layers.Layer):
    def __init__(self, filters, num_heads=8):
        super(MultiHeadAttentionLayer, self).__init__()
        assert filters % num_heads == 0, "filters must be divisible by num_heads"
        self.filters = filters
        self.num_heads = num_heads
        self.depth = filters // num_heads

        # Define query, key, value convolutions
        self.query_conv = layers.Conv2D(filters, kernel_size=1, dtype='float32')
        self.key_conv = layers.Conv2D(filters, kernel_size=1, dtype='float32')
        self.value_conv = layers.Conv2D(filters, kernel_size=1, dtype='float32')

        # Gamma parameter for residual learning
        self.gamma = tf.Variable(0.0, dtype='float32')

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth)"""
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, x):
        batch_size = tf.shape(x)[0]  # Dynamic batch size
        residual = x  # Skip connection

        # Generate query, key, and value feature maps
        query = self.query_conv(x)
        key = self.key_conv(x)
        value = self.value_conv(x)

        # Split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Calculate attention scores
        scores = tf.matmul(query, key, transpose_b=True)
        scores = tf.nn.softmax(scores, axis=-1)

        # Apply the attention scores to the value map
        out = tf.matmul(scores, value)
        out = tf.transpose(out, perm=[0, 2, 1, 3])
        out = tf.reshape(out, (batch_size, tf.shape(x)[1], tf.shape(x)[2], self.filters))

        # Add the attention results to the original feature maps
        out = self.gamma * out + residual
        return out


In [9]:
def residual_block(x, filters, kernel_size=3, stride=1):
    """Residual Block with skip connection"""
    shortcut = x
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, kernel_size, strides=1, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    return layers.ReLU()(x)

def build_attention_cnn(input_shape=(512, 512, 3), num_classes=8):
    inputs = layers.Input(shape=input_shape, dtype='float32')

    # Convolutional Block 1
    x = layers.Conv2D(32, 3, strides=2, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)

    # Convolutional Block 2
    x = residual_block(x, 64, stride=2)

    # Convolutional Block 3
    x = residual_block(x, 128, stride=2)

    # Add Multi-Head Attention Layer with Skip Connections
    x = MultiHeadAttentionLayer(128, num_heads=8)(x)

    # Convolutional Block 4
    x = residual_block(x, 256, stride=2)

    # Global Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Dense Classification Head
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    predictions = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, predictions)
    return model

# Instantiate and compile the multi-head attention-enhanced CNN
model = build_attention_cnn()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 256, 256, 32) 896         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_23 (BatchNo (None, 256, 256, 32) 128         conv2d_44[0][0]                  
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 128, 128, 64) 18496       batch_normalization_23[0][0]     
____________________________________________________________________________________________

In [6]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#     except RuntimeError as e:
#         print(e)


In [13]:
# Data augmentation and preprocessing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load and prepare the data
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(180, 180),
    batch_size=32,
    class_mode='categorical'
)



Found 9376 images belonging to 8 classes.
Found 2344 images belonging to 8 classes.


In [14]:
sample_images, sample_labels = next(train_generator)
print("Sample Labels (First 5):", sample_labels[:5])

Sample Labels (First 5): [[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]


In [15]:
import tensorflow as tf

# Check available devices
print("Available GPU devices:", tf.config.experimental.list_physical_devices('GPU'))


Available GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [16]:
# Train the multi-head attention-enhanced CNN model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32
)
# 293/293 [==============================] - 103s 350ms/step - loss: 1.0234 - accuracy: 0.6448 - val_loss: 1.8943 - val_accuracy: 0.1396


Epoch 1/30
293/293 [==============================] - 201s 653ms/step - loss: 0.9986 - accuracy: 0.6555 - val_loss: 1.4851 - val_accuracy: 0.4229
Epoch 2/30
293/293 [==============================] - 230s 785ms/step - loss: 0.8833 - accuracy: 0.6804 - val_loss: 1.0207 - val_accuracy: 0.6704
Epoch 3/30
293/293 [==============================] - 191s 650ms/step - loss: 0.8299 - accuracy: 0.6981 - val_loss: 0.9599 - val_accuracy: 0.6866
Epoch 4/30
293/293 [==============================] - 218s 744ms/step - loss: 0.8057 - accuracy: 0.7051 - val_loss: 0.7855 - val_accuracy: 0.7102
Epoch 5/30
293/293 [==============================] - 213s 727ms/step - loss: 0.7734 - accuracy: 0.7170 - val_loss: 0.8422 - val_accuracy: 0.7012
Epoch 6/30
293/293 [==============================] - 212s 722ms/step - loss: 0.7605 - accuracy: 0.7214 - val_loss: 0.8247 - val_accuracy: 0.7029
Epoch 7/30
293/293 [==============================] - 210s 717ms/step - loss: 0.7423 - accuracy: 0.7274 - val_loss: 0.9409 -